In [5]:
from __future__ import absolute_import
from __future__ import print_function

from data_utils import load_task, vectorize_data
import sklearn
from memn2n import MemN2N
from itertools import chain
from six.moves import range, reduce

import tensorflow as tf
import numpy as np
import pandas as pd

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

NameError: name 'tf' is not defined

In [3]:
tf.app.flags.DEFINE_float("learning_rate", 0.01, "Learning rate for SGD.")
tf.app.flags.DEFINE_float("anneal_rate", 25, "Number of epochs between halving the learnign rate.")
tf.app.flags.DEFINE_float("anneal_stop_epoch", 100, "Epoch number to end annealed lr schedule.")
tf.app.flags.DEFINE_float("max_grad_norm", 40.0, "Clip gradients to this norm.")
tf.app.flags.DEFINE_integer("evaluation_interval", 10, "Evaluate and print results every x epochs")
tf.app.flags.DEFINE_integer("batch_size", 32, "Batch size for training.")
tf.app.flags.DEFINE_integer("hops", 3, "Number of hops in the Memory Network.")
tf.app.flags.DEFINE_integer("epochs", 100, "Number of epochs to train for.")
tf.app.flags.DEFINE_integer("embedding_size", 20, "Embedding size for embedding matrices.")
tf.app.flags.DEFINE_integer("memory_size", 50, "Maximum size of memory.")
tf.app.flags.DEFINE_integer("task_id", 1, "bAbI task id, 1 <= id <= 20")
tf.app.flags.DEFINE_integer("random_state", None, "Random state.")
tf.app.flags.DEFINE_string("data_dir", "data/tasks_1-20_v1-2/en/", "Directory containing bAbI tasks")
FLAGS = tf.app.flags.FLAGS

print("Started Task:", FLAGS.task_id)

Started Task: 1


# LOAD AND PROCESS DATA

In [4]:
x = open("train_bodies.txt", "r").readlines()

In [5]:
y = open("train_claims.txt", "r").readlines()

```
0 - unrelated
1 - discussed
2 - agree
3 - disagree
```

In [6]:
x[0]

"0 a small meteorite crashed into a wooded area in nicaragua 's capital of managua overnight the government said sunday residents reported hearing a mysterious boom that left a <number>-foot deep crater near the city 's airport the associated press reports\n"

In [7]:
y[0]

'712 police find mass graves with at least <number> bodies  near mexico town where <number> students disappeared after police clash 0\n'

In [8]:
x_proc = []

In [9]:
for s in x:
    s_proc = s.split(" ")
    s_init = []
    sent = ' '.join(s_proc[1:])
    sent = sent.split("\n")[0]
    s_init.extend([s_proc[0], sent])
    x_proc.append(s_init)

In [10]:
x_proc[0]

['0',
 "a small meteorite crashed into a wooded area in nicaragua 's capital of managua overnight the government said sunday residents reported hearing a mysterious boom that left a <number>-foot deep crater near the city 's airport the associated press reports"]

In [11]:
paras = np.array(x_proc)

In [12]:
len(np.unique(paras[:,0]))

1683

In [13]:
y_proc = []
for s in y:
    s_proc = s.split("\n")[0].split(" ")
    s_init = []
    s_init.extend([s_proc[0], ' '.join(s_proc[1:-1]), s_proc[-1]])
    y_proc.append(s_init)

In [14]:
y_proc[0]

['712',
 'police find mass graves with at least <number> bodies  near mexico town where <number> students disappeared after police clash',
 '0']

In [15]:
claims = np.array(y_proc)

In [16]:
len(np.unique(claims[:,0])) == len(np.unique(paras[:,0]))

True

In [17]:
claimsdf = pd.DataFrame(claims)

In [18]:
parasdf = pd.DataFrame(paras)

In [19]:
parasdf.head()

,0,1
0,0,a small meteorite crashed into a wooded area i...
1,0,government spokeswoman rosario murillo said a ...
2,0,the crater left by the meteorite had a radius ...
3,0,humberto garcia of the astronomy center at the...
4,0,we have to study it more because it could be i...


In [20]:
claimsdf.head()

,0,1,2
0,712,police find mass graves with at least <number>...,0
1,158,hundreds of palestinians flee floods in gaza a...,2
2,137,christian bale passes on role of steve jobs ac...,0
3,1034,hbo and apple in talks for <number>month apple...,0
4,1923,spider burrowed through tourist 's stomach and...,3


In [21]:
ours = []

dict_stances = {
    '0': "unrelated",
    '1': "discussed",
    '2': "agree",
    '3': "disagree"
}

for bodyid in parasdf[0].unique():
    paras_with_this_id = list(parasdf[parasdf[0]==bodyid][1].values)
    paras_with_this_id = [p.split() for p in paras_with_this_id]
    claims_for_this_id = claimsdf[claimsdf[0]==bodyid]
    for i, claim in claims_for_this_id.iterrows():
        entity = []
        entity.append(list(paras_with_this_id))
        entity.append(claim[1].split(" "))
        claim_int = claim[2]
        claim_text = dict_stances[claim_int]
        entity.append([claim_text])
        ours.append(entity)

In [22]:
data = ours

# VOCAB, VECTORIZATION

In [23]:
vocab = sorted(reduce(lambda x, y: x | y, (set(list(chain.from_iterable(s)) + q + a) for s, q, a in data)))
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))

In [24]:
max_story_size = max(map(len, (s for s, _, _ in data)))
mean_story_size = int(np.mean([ len(s) for s, _, _ in data ]))
sentence_size = max(map(len, chain.from_iterable(s for s, _, _ in data)))
query_size = max(map(len, (q for _, q, _ in data)))
# memory_size = min(FLAGS.memory_size, max_story_size)
memory_size = max_story_size

In [25]:
# Add time words/indexes
for i in range(memory_size):
    word_idx['time{}'.format(i+1)] = 'time{}'.format(i+1)

In [26]:
vocab_size = len(word_idx) + 1 # +1 for nil word
sentence_size = max(query_size, sentence_size) # for the position
sentence_size += 1  # +1 for time words

print("Longest sentence length", sentence_size)
print("Longest story length", max_story_size)
print("Average story length", mean_story_size)

Longest sentence length 797
Longest story length 9
Average story length 6


In [27]:
data = data[:5000]

In [28]:
# train/validation/test sets
S, Q, A = vectorize_data(data, word_idx, sentence_size, memory_size)

# SPLIT

In [29]:
trainS, testS, trainQ, testQ, trainA, testA = cross_validation.train_test_split(S, Q, A, test_size=.2, random_state=FLAGS.random_state)

We split the full dataset into a train and test. Then we split the test again to obtain a validation set.

In [30]:
testS, valS, testQ, valQ, testA, valA = cross_validation.train_test_split(testS, testQ, testA, test_size=.5, random_state=FLAGS.random_state)

In [31]:
print(testS[0])

print("Training set shape", trainS.shape)

# params
n_train = trainS.shape[0]
n_test = testS.shape[0]
n_val = valS.shape[0]

print("Training Size", n_train)
print("Validation Size", n_val)
print("Testing Size", n_test)

[[  145 16833  8254 ...     0     0 20438]
 [ 2828 13054  9519 ...     0     0 20437]
 [ 2829 11214  9519 ...     0     0 20436]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]
Training set shape (4000, 9, 797)
Training Size 4000
Validation Size 500
Testing Size 500


In [32]:
train_labels = np.argmax(trainA, axis=1)
test_labels = np.argmax(testA, axis=1)
val_labels = np.argmax(valA, axis=1)

# MODEL

In [33]:
tf.set_random_seed(FLAGS.random_state)
batch_size = FLAGS.batch_size

In [34]:
batches = zip(range(0, n_train-batch_size, batch_size), range(batch_size, n_train, batch_size))
batches = [(start, end) for start, end in batches]

In [ ]:
with tf.Session() as sess:
    model = MemN2N(batch_size, vocab_size, sentence_size, memory_size, 
                   FLAGS.embedding_size, 
                   session=sess,
                   hops=FLAGS.hops, 
                   max_grad_norm=FLAGS.max_grad_norm)
    for t in range(1, FLAGS.epochs+1):
        # Stepped learning rate
        if t - 1 <= FLAGS.anneal_stop_epoch:
            anneal = 2.0 ** ((t - 1) // FLAGS.anneal_rate)
        else:
            anneal = 2.0 ** (FLAGS.anneal_stop_epoch // FLAGS.anneal_rate)
        lr = FLAGS.learning_rate / anneal

        np.random.shuffle(batches)
        total_cost = 0.0
        for start, end in batches:
            s = trainS[start:end]
            q = trainQ[start:end]
            a = trainA[start:end]
            cost_t = model.batch_fit(s, q, a, lr)
            total_cost += cost_t

        if t % FLAGS.evaluation_interval == 0:
            train_preds = []
            for start in range(0, n_train, batch_size):
                end = start + batch_size
                s = trainS[start:end]
                q = trainQ[start:end]
                pred = model.predict(s, q)
                train_preds += list(pred)

            val_preds = model.predict(valS, valQ)
            train_acc = metrics.accuracy_score(np.array(train_preds), train_labels)
            val_acc = metrics.accuracy_score(val_preds, val_labels)

            print('-----------------------')
            print('Epoch', t)
            print('Total Cost:', total_cost)
            print('Training Accuracy:', train_acc)
            print('Validation Accuracy:', val_acc)
            print('-----------------------')

    test_preds = model.predict(testS, testQ)
    test_acc = metrics.accuracy_score(test_preds, test_labels)
    print("Testing Accuracy:", test_acc)

In [46]:
len(word_idx.items())

20443

In [59]:
idx2word = {v:k for k,v in word_idx.items()}

In [60]:
[idx2word[k] for k in trainS[0][0]]

KeyError: 20441